In [7]:
import nltk
import pandas as pd
import numpy as np
import os, sys
PATH = os.path.abspath("")[:-3]

In [3]:
# SOLO HACER LA PRIMERA VEZ QUE SE TIRA
# Se va a abrir en una ventana aparte de python, le das a download todo y cuando acabe la cierras.
# Si todo ha salido guay devuelve True.
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# Preparar datos para predecir un usuario

In [112]:

# SCRAPPEAR UN USUARIO
from scrap import scrap_user

USER = "ComicGirlAshley"
tweets = scrap_user(USER)
tweets.to_csv(PATH+"data/example.csv", index=False) # Se guardan los tweets en un csv pero no hace falta lol.

In [113]:
# CÁLCULO DEL FACTOR REP
from sklearn.feature_extraction.text import CountVectorizer
tweets = pd.read_csv(PATH+"data/example.csv") # Se puede leer el csv de antes.
contents = tweets["content"]
toks = []
sw = nltk.corpus.stopwords
stemmer = nltk.stem.snowball.EnglishStemmer()
# Tokenizar, stemmear, filtrar stop-words.
for content in contents.values:
    tk = nltk.word_tokenize(content)
    filtered_sentence = [w for w in tk if not w.lower() in sw.words('english')]
    toks.append([stemmer.stem(word) for word in filtered_sentence])
# Se cuenta número de repeticiones de palabras y se suman.
try:
    vect = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False, ngram_range=(1, 15), min_df=0.1).fit_transform(toks)
    factor_rep = vect.sum()
except:
    factor_rep = 0
print(factor_rep)

305


C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [114]:
# REPETICIÓN DE LAS FECHAS
from datetime import datetime
date = tweets["date"]
hours = []
for h in date.values:
    hours.append(pd.to_datetime(h).hour)
valores_finales=[]
tot= len(hours)
result = dict()
#contando el numero de repeticiones
for j in hours:
    if j not in result:
        result[j] = 0 
    result[j] += 1

#viendo el mayor valor y computando porcentaje
values=[]
mayor=0
for dato, valor in result.items():
    if valor > mayor:
        mayor= valor
hora_rep = mayor/tot
print(hora_rep)

0.13725490196078433


In [115]:
# DATOS MEDIOS
avg_likes = np.average(tweets["likeCount"].values)
avg_retweets = np.average(tweets["retweetCount"].values)
avg_reply = np.average(tweets["replyCount"].values)
avg_quote = np.average(tweets["quoteCount"].values)

In [116]:
# El usuario listo para predecir se guarda en user_data.
user_data = tweets[["verified", "followersCount", "friendsCount", "tweetsCount", "listedCount", "mediaCount"]].iloc[-1, :].values.tolist()
user_data.insert(0, factor_rep)
user_data.extend([avg_reply, avg_retweets, avg_likes, avg_quote, hora_rep, user_data[2]-user_data[3]])
print(user_data)
print(len(user_data))

[305, False, 26662, 98, 20070, 133, 6288, 3.8529411764705883, 16.38235294117647, 235.84313725490196, 0.8725490196078431, 0.13725490196078433, 26564]
13


# Modelo

In [61]:
# Se leen los datos generados con model.py
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
training = pd.read_csv(PATH+"data/training_new.csv").sample(frac=1).reset_index(drop=True)
training["clase"].replace("bot",1, inplace= True)
training["clase"].replace("human",0, inplace= True)
training.pop("username")
label = training.pop("clase")
scaler= StandardScaler()
data_s = scaler.fit_transform(training)
scaler_m= MinMaxScaler()
data_m = scaler.fit_transform(training)
print(data)
print(label)

[[-0.13108252  2.33066514 -0.0695483  ... -0.08192625 -0.07217462
   0.06954871]
 [-0.19448928  2.33066514  1.18894896 ... -0.05890278 -0.49191673
  -1.18894861]
 [ 0.05193242 -0.42906207 -0.13150805 ... -0.08820538 -0.07747291
   0.13150798]
 ...
 [-0.18151971 -0.42906207 -0.13144538 ... -0.08820538 -0.61545287
   0.13144531]
 [-0.06719542  2.33066514 -0.13135763 ... -0.08820538 -0.74887189
   0.13135805]
 [ 0.06153951  2.33066514  0.4834805  ... -0.02698387 -0.084809
  -0.48348012]]
0       1
1       1
2       0
3       1
4       1
       ..
2407    1
2408    0
2409    1
2410    1
2411    0
Name: clase, Length: 2412, dtype: int64


## probar alg

In [58]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import xgboost as xgb


regre

In [73]:
data= data_m
x_train, x_test, y_train, y_test = train_test_split(data, label)

parameters= {"solver":["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"]}
reg= LogisticRegression()

reg_n = GridSearchCV(reg, parameters, cv=5)
reg_n.fit(x_train, y_train)
print(reg_n.best_params_)
print(reg_n.best_score_)

'''
standard:
{'solver': 'sag'}
0.7136560505654949
mi:
{'solver': 'lbfgs'}
0.7086867357401938
'''

C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWa

{'solver': 'lbfgs'}
0.7086867357401938


C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


"\nstandard:\n{'solver': 'sag'}\n0.7136560505654949\nmi:\n{'booster': 'gbtree', 'nthread': 1}\n0.8087265269891798\n"

In [74]:
data= data_s
skf= StratifiedKFold(n_splits=4)
precision=[]
sens=[]
spec=[]
acu=[]

for train_index, test_index in skf.split(data, label):
    x_train= data[train_index]
    y_train = label[train_index]
    x_test= data[test_index]
    y_test = label[test_index]

    reg = LogisticRegression(solver= "sag").fit(x_train, y_train)
    y_prep = reg.predict(x_test)
    acu.append(reg.score(x_test, y_test))
    tn, fp, fn, tp= metrics.confusion_matrix(y_test, y_prep).ravel()
    precision.append(tp/(tp+fp))
    sens.append(tp/(tp+fn))
    spec.append(tn/(tn+fp))

print("acurracy", np.mean(acu))
print("precision" ,np.mean(precision))
print("sens", np.mean(sens))
print("spec", np.mean(spec))

'''
acurracy 0.7101990049751243
precision 0.6736761164619097
sens 0.9750772953897953
spec 0.34062954531082024'''

acurracy 0.7101990049751243
precision 0.6736761164619097
sens 0.9750772953897953
spec 0.34062954531082024


C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWa

xgb

In [65]:
data= data_s
x_train, x_test, y_train, y_test = train_test_split(data, label)

parameters= {"booster": ["gbtree", "gblinear", "dart"], "nthread": range(1,32,2)}
xgb_= xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")

xgb_n = GridSearchCV(xgb_, parameters, cv=5)
xgb_n.fit(x_train, y_train)
print(xgb_n.best_params_)
print(xgb_n.best_score_)

'''
standard:
{'booster': 'gbtree', 'nthread': 1}
0.8065288256990251
mi:
{'booster': 'gbtree', 'nthread': 1}
0.8087265269891798
'''

C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.p

{'booster': 'gbtree', 'nthread': 1}
0.8065288256990251


"\nstandard:\n{'activation': 'relu', 'learning_rate': 'adaptive'}\n0.7827459022665707\nmi:\n{'booster': 'gbtree', 'nthread': 1}\n0.8087265269891798\n"

In [75]:
data= data_m
skf= StratifiedKFold(n_splits=4)
precision=[]
sens=[]
spec=[]
acu=[]

for train_index, test_index in skf.split(data, label):
    x_train= data[train_index]
    y_train = label[train_index]
    x_test= data[test_index]
    y_test = label[test_index]

    xgb_= xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", booster= "gbtree", nthread= 1).fit(x_train,y_train)
    y_prep = xgb_.predict(x_test)
    acu.append(xgb_.score(x_test, y_test))
    tn, fp, fn, tp= metrics.confusion_matrix(y_test, y_prep).ravel()
    precision.append(tp/(tp+fp))
    sens.append(tp/(tp+fn))
    spec.append(tn/(tn+fp))

print("acurracy", np.mean(acu))
print("precision" ,np.mean(precision))
print("sens", np.mean(sens))
print("spec", np.mean(spec))

'''
acurracy 0.8146766169154229
precision 0.8303567912649471
sens 0.8569367553742553
spec 0.7556994245241258
'''

C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.p

acurracy 0.8146766169154229
precision 0.8303567912649471
sens 0.8569367553742553
spec 0.7556994245241258


'\nacurracy 0.8146766169154229\nprecision 0.8303567912649471\nsens 0.8569367553742553\nspec 0.7556994245241258\n'

neuronas

In [52]:

data= data_m
x_train, x_test, y_train, y_test = train_test_split(data, label)

parameters= {"activation": ["identity", "logistic", "tanh", "relu"], "learning_rate":["constant,invscaling", "adaptive"]}
ne= MLPClassifier(max_iter=10000)

ne_n = GridSearchCV(model, parameters, cv=5)
ne_n.fit(x_train, y_train)
print(ne_n.best_params_)
print(ne_n.best_score_)

'''
standard:
{'activation': 'relu', 'learning_rate': 'adaptive'}
0.7827459022665707
mi:
{'activation': 'relu', 'learning_rate': 'adaptive'}
0.7844201955892931
'''

C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Mar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\P

{'activation': 'relu', 'learning_rate': 'adaptive'}
0.7844201955892931


"\nstandard:\n{'activation': 'relu', 'learning_rate': 'adaptive'}\n0.7827459022665707\n"

In [53]:
data= data_m
skf= StratifiedKFold(n_splits=4)
precision=[]
sens=[]
spec=[]
acu=[]

for train_index, test_index in skf.split(data, label):
    x_train= data[train_index]
    y_train = label[train_index]
    x_test= data[test_index]
    y_test = label[test_index]

    ne = MLPClassifier(activation="relu",learning_rate="adaptive", max_iter=10000).fit(x_train,y_train)
    y_prep = ne.predict(x_test)
    acu.append(ne.score(x_test, y_test))
    tn, fp, fn, tp= metrics.confusion_matrix(y_test, y_prep).ravel()
    precision.append(tp/(tp+fp))
    sens.append(tp/(tp+fn))
    spec.append(tn/(tn+fp))

print("acurracy", np.mean(acu))
print("precision" ,np.mean(precision))
print("sens", np.mean(sens))
print("spec", np.mean(spec))

'''
acurracy 0.7645107794361525
precision 0.7549646432499513
sens 0.8832718207718208
spec 0.5988387718965409

'''

acurracy 0.7645107794361525
precision 0.7549646432499513
sens 0.8832718207718208
spec 0.5988387718965409


probar 

In [130]:
#transformando los datos del crapeado a la normalizacion necesaria y se selecciona el mejor modelo

model= xgb_
scaler_m= MinMaxScaler()
user_d=np.array(user_data)
user_array= user_d.reshape(-1, 1)
user_array = scaler.fit_transform(user_array)

final= []
for i in range(0,13):
    final.append(float(user_array[i]))


[[-0.57423662]
 [-0.60403359]
 [ 2.00070951]
 [-0.59445948]
 [ 1.35670431]
 [-0.59104016]
 [ 0.01027234]
 [-0.60365717]
 [-0.60243311]
 [-0.5809929 ]
 [-0.60394834]
 [-0.60402018]
 [ 1.9911354 ]]


[-0.5742366232413479,
 -0.6040335868335599,
 2.0007095059387505,
 -0.5944594804990131,
 1.356704312496583,
 -0.5910401568081035,
 0.010272337978993056,
 -0.6036571738894346,
 -0.602433113475714,
 -0.5809929003605849,
 -0.6039483431897249,
 -0.6040201777210465,
 1.9911353996042038]

In [132]:
# Imprime la probabilidad de que el usuario preparado antes sea bot.
print(model.predict_proba(np.array([final]))[0])
result= model.predict(np.array([final]))[0]
if result==1:
    print("bot")
else:
    print("human")


[0.01588929 0.9841107 ]
bot
